# Notebook 04: Testes de Queries RAG

Este notebook testa o sistema RAG completo. Aqui vamos:

1. **Testar** queries de busca no Pinecone
2. **Validar** qualidade da recuperação de contexto
3. **Exemplificar** uso prático do RAG
4. **Avaliar** métricas de qualidade

## 📋 Pré-requisitos

- Notebook 03 executado com sucesso (dados ingeridos no Pinecone)
- Variáveis de ambiente configuradas
- Dependências instaladas

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo).


In [ ]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================

import sys
from pathlib import Path

# Adiciona o diretório raiz ao path
root_dir = Path.cwd().parent
sys.path.insert(0, str(root_dir))

# Importa módulos do pipeline RAG
from scripts.embeddings_manager import EmbeddingsManager
from scripts.rag_query import (
    query_medical_rag,
    format_context_for_llm,
    get_unique_articles,
    filter_by_score
)
from config.settings import get_settings

# Carrega configurações
settings = get_settings()

print("✅ Bibliotecas importadas com sucesso!")
settings.print_config()


In [ ]:
# ============================================================================
# ETAPA 1: INICIALIZAÇÃO DO GERENCIADOR DE EMBEDDINGS
# ============================================================================

print("=" * 80)
print("🔧 CONFIGURANDO EMBEDDINGS")
print("=" * 80)

embeddings_manager = EmbeddingsManager()

print(f"Provider: {embeddings_manager.provider}")
print(f"Model: {embeddings_manager.model_name}")
print(f"Dimensão: {embeddings_manager.get_embedding_dimension()}")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 2: QUERY BÁSICA DE TESTE
# ============================================================================
# Testa uma query simples para verificar se o sistema está funcionando

print("=" * 80)
print("🔍 TESTE 1: QUERY BÁSICA")
print("=" * 80)

query = "Do mitochondria play a role in cell death?"
print(f"Query: '{query}'")
print("-" * 80)

# Executa query
results = query_medical_rag(
    query,
    embeddings_manager=embeddings_manager,
    top_k=5
)

print(f"\n✅ Query executada com sucesso!")
print(f"   Resultados encontrados: {len(results)}")
print("\n📄 Resultados:")

for i, result in enumerate(results, 1):
    print(f"\n   Resultado {i}:")
    print(f"   - Article ID: {result['article_id']}")
    print(f"   - Score: {result['score']:.3f}")
    print(f"   - Chunk Index: {result['chunk_index']}")
    print(f"   - Texto (primeiros 150 caracteres):")
    print(f"     {result['text'][:150]}...")

print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 3: QUERIES MÉDICAS ESPECÍFICAS
# ============================================================================
# Testa queries com diferentes temas médicos

print("=" * 80)
print("🔍 TESTE 2: QUERIES MÉDICAS ESPECÍFICAS")
print("=" * 80)

# Lista de queries de teste
test_queries = [
    "What is the role of mitochondria in apoptosis?",
    "How does visual acuity testing work?",
    "What are the symptoms of diabetes?",
    "How is cancer treated?",
    "What causes heart disease?",
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query: '{query}'")
    print("-" * 80)
    
    results = query_medical_rag(
        query,
        embeddings_manager=embeddings_manager,
        top_k=3
    )
    
    print(f"Resultados: {len(results)}")
    
    if results:
        # Mostra o melhor resultado
        best = results[0]
        print(f"Melhor resultado (Score: {best['score']:.3f}):")
        print(f"  Article ID: {best['article_id']}")
        print(f"  Texto: {best['text'][:200]}...")
    else:
        print("⚠️  Nenhum resultado encontrado")

print("\n" + "=" * 80)


In [ ]:
# ============================================================================
# ETAPA 4: FORMATAÇÃO DE CONTEXTO PARA LLM
# ============================================================================
# Demonstra como formatar resultados para uso com LLM

print("=" * 80)
print("📝 TESTE 3: FORMATAÇÃO DE CONTEXTO PARA LLM")
print("=" * 80)

query = "mitochondria and programmed cell death"
print(f"Query: '{query}'")
print("-" * 80)

# Busca contexto
results = query_medical_rag(
    query,
    embeddings_manager=embeddings_manager,
    top_k=5
)

# Formata contexto para LLM
context = format_context_for_llm(results)

print("\n📄 Contexto formatado para LLM:")
print("-" * 80)
print(context)
print("-" * 80)

print(f"\n✅ Contexto formatado com {len(results)} resultados")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 5: FILTRAGEM POR SCORE
# ============================================================================
# Demonstra como filtrar resultados por score mínimo de similaridade

print("=" * 80)
print("🎯 TESTE 4: FILTRAGEM POR SCORE")
print("=" * 80)

query = "cell death mechanisms"
print(f"Query: '{query}'")
print("-" * 80)

# Busca com mais resultados
results = query_medical_rag(
    query,
    embeddings_manager=embeddings_manager,
    top_k=10
)

print(f"Total de resultados: {len(results)}")

# Filtra por score mínimo
min_score = 0.7
filtered_results = filter_by_score(results, min_score=min_score)

print(f"\nResultados com score >= {min_score}: {len(filtered_results)}")
print("\nDistribuição de scores:")
for i, result in enumerate(results[:5], 1):
    print(f"  {i}. Score: {result['score']:.3f}")

print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 6: ANÁLISE DE ARTIGOS ÚNICOS
# ============================================================================
# Analisa quantos artigos únicos foram recuperados

print("=" * 80)
print("📊 TESTE 5: ANÁLISE DE ARTIGOS ÚNICOS")
print("=" * 80)

query = "medical research findings"
print(f"Query: '{query}'")
print("-" * 80)

results = query_medical_rag(
    query,
    embeddings_manager=embeddings_manager,
    top_k=10
)

# Obtém artigos únicos
unique_articles = get_unique_articles(results)

print(f"Total de resultados: {len(results)}")
print(f"Artigos únicos: {len(unique_articles)}")
print(f"Diversidade: {len(unique_articles)/len(results)*100:.1f}%")
print(f"\nIDs dos artigos únicos:")
for article_id in unique_articles[:10]:
    print(f"  - {article_id}")

print("=" * 80)


## ✅ Conclusão da Etapa 4

Neste notebook você:
- ✅ Testou queries básicas no sistema RAG
- ✅ Validou recuperação de contexto relevante
- ✅ Formatou contexto para uso com LLM
- ✅ Analisou qualidade dos resultados
- ✅ Explorou funcionalidades avançadas (filtros, análise)

## 🎉 Pipeline RAG Completo!

O sistema RAG está funcionando e pronto para uso! Você pode agora:
- Integrar com seu backend (Node.js)
- Usar em aplicações de assistente médico
- Expandir com mais dados ou funcionalidades

## 📌 Próximos Passos Sugeridos

1. **Integração com Backend**: Use os scripts Python ou integre diretamente com Node.js
2. **Otimização**: Ajuste chunk_size, top_k, e outros parâmetros conforme necessário
3. **Monitoramento**: Implemente logging e métricas de uso
4. **Expansão**: Adicione mais dados médicos ao Pinecone
